# 使用Snscrape+多线程爬取推文

In [1]:
import pandas as pd
import datetime
import os
import snscrape.modules.twitter as sntwitter
import numpy as np
import time
import random
import sys
import queue
import threading
from IPython.display import clear_output as clear

Tweet_id存储在年月格式的文件夹中，名称有固定格式，按照解析式列出文件名并选出实际存在的文件名

In [2]:
date_series = [f"D:\\hrfan\\RussoUkrainianWar_Dataset-main\\2022-0{month}\\tweet_ids_day_2022-{month}-{date}.txt"
              for month in range(2,10) for date in range(1,32)]
dates = []
for date_data in date_series:
    if os.path.exists(date_data):
        with open(date_data,'r') as f:
            dates.append(date_data)

In [3]:
dates = dates[20:]

In [4]:
sem=threading.BoundedSemaphore(42) #限制线程的最大数量42，可以根据电脑性能调整线程数
def tweetscraper(j, tweets_list):
    with sem:
        clear(wait=True)#先清除输出区域，不然会一直打印造成ipynb文件过大
        try:
            for i,tweet in enumerate(sntwitter.TwitterTweetScraper(j).get_items()):
                tweets_list.append([
                    #这里有些用的是老版本的变量名，会提示使用新的名称，但由于我最开始就用老名称爬就没改
                    tweet.date, tweet.url, tweet.id, tweet.content, tweet.replyCount, tweet.retweetCount, tweet.likeCount,
                    tweet.quoteCount, tweet.conversationId, tweet.lang, tweet.source, tweet.sourceUrl, tweet.outlinks, tweet.media,
                    tweet.retweetedTweet, tweet.quotedTweet, tweet.inReplyToTweetId, tweet.inReplyToUser, tweet.mentionedUsers, 
                    tweet.coordinates, tweet.place, tweet.hashtags, tweet.cashtags, #tweet相关的字段
                    tweet.user.username, tweet.user.displayname, tweet.user.id, tweet.user.rawDescription, tweet.user.descriptionUrls,
                    tweet.user.verified, tweet.user.created,tweet.user.followersCount, tweet.user.friendsCount, tweet.user.statusesCount, 
                    tweet.user.favouritesCount,tweet.user.listedCount, tweet.user.mediaCount, tweet.user.location, tweet.user.protected,
                    tweet.user.linkUrl,tweet.user.profileImageUrl, tweet.user.profileBannerUrl, tweet.user.label ##user相关的字段
                    ])
            time.sleep(1)
            #print(len(tweets_list), "tweets have been scraped", flush = True)
        except:
            print("a mistake have been passed", flush = True)
            time.sleep(3)#多休息一下

In [ ]:
#开始爬！
#ver1.0：https://blog.wangtwothree.com/code/196.html 多线程的内存占用问题
#ver2.0：还是出现崩溃现象，https://stackoverflow.com/questions/69005764/iostream-flush-timed-out-errors-when-multithreading
#找到了和自己情况一样的提问，但没有找到解决方案。重写代码以实现把一个文件分成好几个来保存，看能不能解决这个问题

for i in range(len(dates)):
    rootname = dates[i] +'_tweets.csv'#之前输出的文件地址
    nametime = 1
    name = dates[i]+'_tweets'+'_'+str(nametime)+'.csv'
    if os.path.exists(rootname):#如果已经爬过了就跳过
        print(rootname, ' has been scraped, passed.', flush=True)
        continue
        
    
        
    else:
        while os.path.exists(name):
            nametime+=1
            name = dates[i]+'_tweets'+'_'+str(nametime)+'.csv'
            
        testset = []
        with open(dates[i], 'r') as f:
            for line in f:
                testset.append(line.strip('\\n').split(','))
        test = []
        
        for j in testset:
            test.append(int(j[0]))
            
        if nametime>1:
            test = test[500000*(nametime-1):]#如果之前已经爬了就跳过
        # create multi-threads to scrape
        threads = []
        tweets_list = []
        print(dates[i], 'scraper begins')

        
        for j in test:
            if (test.index(j)+1)%500000!=0:#每50万条保存一个子csv文件
                t = threading.Thread(target = tweetscraper, args=(j, tweets_list))
                t.start()
                threads.append(t)
            else:
                for t in threads:
                    t.join()
                    threads.remove(t)#为了内存稳定，需要remove之前的线程，否则数据过多的时候会卡死，功亏一篑。加入这行代码后内存占用有改善但还是没用
                name = dates[i]+'_tweets'+'_'+str(nametime)+'.csv'
                nametime+=1
                tweets_df = pd.DataFrame(tweets_list, columns=[
                                                    'Datetime', 'Tweet_url', 'Tweet_id', 'Tweet_content', 'Tweet_reply_count', 'Tweet_retweet_count',
                                                    'Tweet_like_count', 'Tweet_quote_count', 'Tweet_conversation_id', 'Tweet_language', 'Tweet_source', 
                                                    'Tweet_source_url', 'Tweet_links', 'Tweet_mdeia', 'Tweet_retweeted_tweet', 'Tweet_quoted_tweet', 
                                                    'Tweet_inReplyToTweetId','Tweet_inReplyToUser','Tweet_mentioned_users','Tweet_coordinates', 'Tweet_place', 
                                                    'Tweet_hashtags', 'Tweet_cashtags',#tweet 相关的字段
                                                    'Username','User_displayname', 'User_id', 'User_profile_description', 'User_description_link', 'User_verified',
                                                    'User_created', 'User_followers_count', 'User_friends_count', 'User_statuses_count', 'User_favourites_count',
                                                    'User_listed_count', 'User_media_count', 'User_location', 'User_protected', 'User_profile_link', 'User_profile_image_url',
                                                    'User_profile_banner_url', 'User_label' #user 相关的字段
                                                  ])

                tweets_df.to_csv(name)
                tweets_list=[]
            
        print(dates[i], 'has been scraped')
        for t in threads:
            t.join()
            threads.remove(t)#为了内存稳定，需要remove之前的线程，否则数据过多的时候会卡死，功亏一篑。加入这行代码后内存占用有改善但还是没用
        
        name = dates[i]+'_tweets'+'_'+str(nametime)+'.csv'
        tweets_df = pd.DataFrame(tweets_list, columns=[
                                                    'Datetime', 'Tweet_url', 'Tweet_id', 'Tweet_content', 'Tweet_reply_count', 'Tweet_retweet_count',
                                                    'Tweet_like_count', 'Tweet_quote_count', 'Tweet_conversation_id', 'Tweet_language', 'Tweet_source', 
                                                    'Tweet_source_url', 'Tweet_links', 'Tweet_mdeia', 'Tweet_retweeted_tweet', 'Tweet_quoted_tweet', 
                                                    'Tweet_inReplyToTweetId','Tweet_inReplyToUser','Tweet_mentioned_users','Tweet_coordinates', 'Tweet_place', 
                                                    'Tweet_hashtags', 'Tweet_cashtags',#tweet 相关的字段
                                                    'Username','User_displayname', 'User_id', 'User_profile_description', 'User_description_link', 'User_verified',
                                                    'User_created', 'User_followers_count', 'User_friends_count', 'User_statuses_count', 'User_favourites_count',
                                                    'User_listed_count', 'User_media_count', 'User_location', 'User_protected', 'User_profile_link', 'User_profile_image_url',
                                                    'User_profile_banner_url', 'User_label' #user 相关的字段
                                                  ])

        tweets_df.to_csv(name)

Error retrieving https://api.twitter.com/1.1/guest/activate.json: non-200 response (429)
4 requests to https://api.twitter.com/1.1/guest/activate.json failed, giving up.
Errors: non-200 response (429), non-200 response (429), non-200 response (429), non-200 response (429)


a mistake have been passed


Error retrieving https://api.twitter.com/1.1/guest/activate.json: non-200 response (429)
4 requests to https://api.twitter.com/1.1/guest/activate.json failed, giving up.
Errors: non-200 response (429), non-200 response (429), non-200 response (429), non-200 response (429)


a mistake have been passed


Error retrieving https://api.twitter.com/1.1/guest/activate.json: non-200 response (429)
4 requests to https://api.twitter.com/1.1/guest/activate.json failed, giving up.
Errors: non-200 response (429), non-200 response (429), non-200 response (429), non-200 response (429)


...

Error retrieving https://api.twitter.com/1.1/guest/activate.json: non-200 response (429)
4 requests to https://api.twitter.com/1.1/guest/activate.json failed, giving up.
Errors: non-200 response (429), non-200 response (429), non-200 response (429), non-200 response (429)


a mistake have been passed


Error retrieving https://api.twitter.com/1.1/guest/activate.json: non-200 response (429)
4 requests to https://api.twitter.com/1.1/guest/activate.json failed, giving up.
Errors: non-200 response (429), non-200 response (429), non-200 response (429), non-200 response (429)


a mistake have been passed


Error retrieving https://api.twitter.com/1.1/guest/activate.json: non-200 response (429)
4 requests to https://api.twitter.com/1.1/guest/activate.json failed, giving up.
Errors: non-200 response (429), non-200 response (429), non-200 response (429), non-200 response (429)
